# Machine learning project

1. Data adquisition: import data, first view of data and curious findings
2. Data check: cleaning NaN, visualize data and check outliers
2. define features 
3. PCA chose features
4. try clasifiers and check scores
5. chose one clasifier and explain result

The purpose of this project is to choose and algorithm and fine tune it to get identification of Person of interest in Enron fraud. It is a supervised learning as we have the solution and we could check the error of the algorithm. Once we get a set of data, the levers that we have to get a good result are the features selection and definition, the classifier and the fine tune of the classifier. 
The key is the data quality and the data treatment before starting to submit the data set to the algorithm. We need to spend some time on data to understand what the matter is and to ensure the information we will pass to the algorithm will not induce bays, error or misleading results
The project result is the identification of who could be considered as person of interest in the investigation of Enron fraud. At the time of the fraud investigation some financial figures from employees were reveal as well as the emails in their accounts. The idea is to use the information related to financial compensations and emails exchange to define who is and who is not a person of interest for the investigation.
The data set contains 146 entries (Poi) and 21 features. The features in the data fall into three major types, namely financial features, email features and POI labels. 
The features are:

1. POI label: [‘poi’] (boolean, represented as integer)

2. financial features: ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees'] (all units are in US dollars)

3. email features: ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi'] (units are generally number of emails messages; notable exception is ‘email_address’, which is a text string)

# 1. DATA ADQUISITION

First thing to do is to create a dictionary with the features we need, convert that dictionary in a data frame and we check the keys and the values of one key.

In [32]:
import os 

os.chdir(r'C:\Users\sanchez_sanc\Desktop\data_analyst\Curso\4_Machine_learning\Curso_PY_3\tools') 

import sys
import pickle
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

#import warnings
#warnings.filterwarnings('ignore')

### Task 1: Select what features we'll use (The first feature must be "poi")
features_list = ['poi','salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus',
                 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses',
                 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 
                 'director_fees']

### Loading the dictionary containing the dataset
os.chdir(r'C:\Users\sanchez_sanc\Desktop\data_analyst\Curso\4_Machine_learning\Curso_PY_3\final_project') # the directory where you want to go

sys.path.append("../Curso_PY_3/final_project/")
data_dict = pickle.load(open("final_project_dataset.pkl", "rb"))

import pandas as pd
data_df = pd.DataFrame.from_dict(data_dict, orient='index')
data_df  = data_df .loc[ :,('poi','salary', 'deferral_payments', 'loan_advances', 'bonus',
                 'deferred_income',  'expenses', 'other', 'long_term_incentive',
                 'director_fees',  'total_payments', 'restricted_stock_deferred','exercised_stock_options', 
                 'restricted_stock', 'total_stock_value', 'email_address', 'from_messages', 'to_messages', 'shared_receipt_with_poi',
            'from_poi_to_this_person', 'from_this_person_to_poi')]
data_df.head()


,poi,salary,deferral_payments,loan_advances,bonus,deferred_income,expenses,other,long_term_incentive,director_fees,...,restricted_stock_deferred,exercised_stock_options,restricted_stock,total_stock_value,email_address,from_messages,to_messages,shared_receipt_with_poi,from_poi_to_this_person,from_this_person_to_poi
METTS MARK,False,365788,NaN,NaN,600000,NaN,94299,1740,NaN,NaN,...,NaN,NaN,585062,585062,mark.metts@enron.com,29,807,702,38,1
BAXTER JOHN C,False,267102,1295738,NaN,1200000,-1386055,11200,2660303,1586055,NaN,...,NaN,6680544,3942714,10623258,NaN,NaN,NaN,NaN,NaN,NaN
ELLIOTT STEVEN,False,170941,NaN,NaN,350000,-400729,78552,12961,NaN,NaN,...,NaN,4890344,1788391,6678735,steven.elliott@enron.com,NaN,NaN,NaN,NaN,NaN
CORDES WILLIAM R,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,651850,386335,1038185,bill.cordes@enron.com,12,764,58,10,0
HANNON KEVIN P,True,243293,NaN,NaN,1500000,-3117011,34039,11350,1617011,NaN,...,NaN,5538001,853064,6391065,kevin.hannon@enron.com,32,1045,1035,32,21


It seams we have several 'NaN' *no information avalaible*. We need to clean those 'NaN' before start to analyse our data
We can see that some people has not email address, but have salary and other financial figures. Lets keep that in mind for later investigation (eg. how many people without adrress mail do we have? have they any specific commmon point? is that an issue when compiling data?)
Let's see a bit the structure, type and content of our data

In [33]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 21 columns):
poi                          146 non-null bool
salary                       146 non-null object
deferral_payments            146 non-null object
loan_advances                146 non-null object
bonus                        146 non-null object
deferred_income              146 non-null object
expenses                     146 non-null object
other                        146 non-null object
long_term_incentive          146 non-null object
director_fees                146 non-null object
total_payments               146 non-null object
restricted_stock_deferred    146 non-null object
exercised_stock_options      146 non-null object
restricted_stock             146 non-null object
total_stock_value            146 non-null object
email_address                146 non-null object
from_messages                146 non-null object
to_messages                  146 non-null object

We have 146 people in our data with 21 posibles informations but we do not know what is the 'NaN' volume per person and per information. We need to be aware of those 'NaN' to be able to correctly interpretate the posible output of our investigation

In [26]:
nan_summary = pd.DataFrame({'size': data_df.count(),
                            'no-nan': data_df.applymap(lambda x: pd.np.nan if x=='NaN' else x).count()},
                            data_df.columns)
nan_summary['nan-proportion'] =1-( nan_summary['no-nan'] / nan_summary['size'])
round((nan_summary.sort_values('nan-proportion',ascending=False)),2)

,size,no-nan,nan-proportion
loan_advances,146,4,0.97
director_fees,146,17,0.88
restricted_stock_deferred,146,18,0.88
deferral_payments,146,39,0.73
deferred_income,146,49,0.66
long_term_incentive,146,66,0.55
bonus,146,82,0.44
from_messages,146,86,0.41
from_poi_to_this_person,146,86,0.41
from_this_person_to_poi,146,86,0.41


Lets think a bit about those 'NaN' before any decission. 
We have a first feature Loan advances that is collect only for 3% of the people. Who they are?
If this feature seams not relevant, we could delete from our investigation. Lets see the details of the payments

In [47]:
loan_people = data_df.loc[:, 'loan_advances'] != 'NaN'
loan_people_df = data_df.loc[loan_people ] 
loan_people = loan_people_df.loc[:, ('poi', 'salary', 'deferral_payments', 'loan_advances', 'bonus',
                 'deferred_income',  'expenses', 'other', 'long_term_incentive',
                 'director_fees',  'total_payments')]
loan_people.head()

,poi,salary,deferral_payments,loan_advances,bonus,deferred_income,expenses,other,long_term_incentive,director_fees,total_payments
LAY KENNETH L,True,1072321,202911,81525000,7000000,-300000,99832,10359729,3600000,NaN,103559793
TOTAL,False,26704229,32083396,83925000,97343619,-27992891,5235198,42667589,48521928,1398517,309886585
PICKERING MARK R,False,655037,NaN,400000,300000,NaN,31653,NaN,NaN,NaN,1386690
FREVERT MARK A,False,1060932,6426990,2000000,2000000,-3367011,86987,7427621,1617011,NaN,17252530


We have found a couple of interesting information
There are 4 rows
1.	There is one entry called TOTAL that is the sum of all values. We should remove this entry as it is not a person 
2.	Kenneth Lay: he got a loan advances of 81m, is nearly 100% of all the loan advances. This advance makes his salary very low. This is misleading information. We should keep that in mind later to create a feature that combine several payments as each payment by itself could be misleading. That will contribute to reduce the amount features the algorithm should analyse and avoid misleading features. We will back later on this
3.	There are two other people who got loan advances that are non Poi that means loan advances is not discriminant of Poi. That could indicate that is not relevant feature by itself.
4.	We do not have 146 useful entries but 145 as one of them is 'Total'

Now less investigate what is behind director fees, as there are only 17 entries. Who are those directors, what is their salay and stock value and the exchange of mails with poi?

In [53]:
d_fees = data_df.loc[:, 'director_fees'] != 'NaN'
d_fees_df = data_df.loc[d_fees ] 
d_fees = d_fees_df.loc[:, ('poi', 'director_fees','salary', 'total_payments','total_stock_value',
                           'email_address', 'from_poi_to_this_person', 'from_this_person_to_poi')]
d_fees

,poi,director_fees,salary,total_payments,total_stock_value,email_address,from_poi_to_this_person,from_this_person_to_poi
CHAN RONNIE,False,98784,NaN,NaN,NaN,NaN,NaN,NaN
BELFER ROBERT,False,3285,NaN,102500,-44093,NaN,NaN,NaN
URQUHART JOHN A,False,36666,NaN,228656,NaN,NaN,NaN,NaN
MENDELSOHN JOHN,False,103750,NaN,148,NaN,NaN,NaN,NaN
WAKEHAM JOHN,False,109298,NaN,213071,NaN,NaN,NaN,NaN
POWERS WILLIAM,False,17500,NaN,NaN,NaN,ken.powers@enron.com,0,0
DUNCAN JOHN H,False,102492,NaN,77492,371750,NaN,NaN,NaN
LEMAISTRE CHARLES,False,112492,NaN,87492,412878,NaN,NaN,NaN
MEYER JEROME J,False,38346,NaN,2151,NaN,NaN,NaN,NaN
PEREIRA PAULO V. FERRAZ,False,101250,NaN,27942,NaN,NaN,NaN,NaN


This is a interesting feature as all the people that got director fees is considered as no Poi, then we will keep this feature as could help the algorithm to discriminate

We can see that despite having director fees, there are two person for which total payment is NaN. 
Lets see what is behind the people with total payment NaN
1. William Power: NaN in this case should be 0 as the director fees are neutralized by deferred income. 
Note: Powers — a lawyer and professor — was tapped by Enron’s board to lead a special investigation to determine what went wrong and why.

2. Ronnie Chan: NaN in this case should be 0 as the director fees are neutralized by deferred income. 
Note: Ronnie Chan Chi-chung is a Hong Kong billionaire businessman. He was a director of Enron Corporation and a member of its audit committee when it filed for bankruptcy as a result of fraud.

Lets see a bit what is behind the entries where total payment is NaN. 

In [29]:
t_payms = data_df.loc[:, 'total_payments'] == 'NaN'
df_t_payms = data_df.loc[t_payms]
df_t_payms

,poi,salary,deferral_payments,loan_advances,bonus,deferred_income,expenses,other,long_term_incentive,director_fees,...,restricted_stock_deferred,exercised_stock_options,restricted_stock,total_stock_value,email_address,from_messages,to_messages,shared_receipt_with_poi,from_poi_to_this_person,from_this_person_to_poi
CORDES WILLIAM R,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,651850,386335,1038185,bill.cordes@enron.com,12,764,58,10,0
LOWRY CHARLES P,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-153686,372205,153686,372205,NaN,NaN,NaN,NaN,NaN,NaN
CHAN RONNIE,False,NaN,NaN,NaN,NaN,-98784,NaN,NaN,NaN,98784,...,-32460,NaN,32460,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WHALEY DAVID A,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,98718,NaN,98718,NaN,NaN,NaN,NaN,NaN,NaN
CLINE KENNETH W,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-472568,NaN,662086,189518,NaN,NaN,NaN,NaN,NaN,NaN
LEWIS RICHARD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,850477,NaN,850477,richard.lewis@enron.com,26,952,739,10,0
POWERS WILLIAM,False,NaN,NaN,NaN,NaN,-17500,NaN,NaN,NaN,17500,...,NaN,NaN,NaN,NaN,ken.powers@enron.com,26,653,12,0,0
PIRO JIM,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,47304,47304,jim.piro@enron.com,16,58,3,0,1
WROBEL BRUCE,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,139130,NaN,139130,NaN,NaN,NaN,NaN,NaN,NaN
MCDONALD REBECCA,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,757301,934065,1691366,rebecca.mcdonald@enron.com,13,894,720,54,1


This is a interesting feature as all the people that total payment NaN is considered as no Poi, then we will keep this feature as could help the algorithm to discriminate

Regarding the mails, we see that there are 111 entries with email address but only 86 entries with values on the messages categories. Let’s see who the people without email is, confirm that they have NaN in message and if they got payments and stock

In [52]:
n_email_people = data_df.loc[:, 'email_address'] == 'NaN'
n_email_people_df = data_df.loc[n_email_people ] 
n_email_people =n_email_people_df.loc[:, ('poi','total_payments','total_stock_value',
                                          'email_address', 'from_messages', 'to_messages' )]
n_email_people.head(len(n_email_people))

,poi,total_payments,total_stock_value,email_address,from_messages,to_messages
BAXTER JOHN C,False,5634343,10623258,NaN,NaN,NaN
NOLES JAMES L,False,774401,368705,NaN,NaN,NaN
LOWRY CHARLES P,False,NaN,372205,NaN,NaN,NaN
WALTERS GARETH W,False,87410,1030329,NaN,NaN,NaN
CHAN RONNIE,False,NaN,NaN,NaN,NaN,NaN
BELFER ROBERT,False,102500,-44093,NaN,NaN,NaN
URQUHART JOHN A,False,228656,NaN,NaN,NaN,NaN
WHALEY DAVID A,False,NaN,98718,NaN,NaN,NaN
MENDELSOHN JOHN,False,148,NaN,NaN,NaN,NaN
CLINE KENNETH W,False,NaN,189518,NaN,NaN,NaN


We have got some interesting information: there is someone called 'THE TRAVEL AGENCY IN THE PARK' and we have 'TOTAL'
Lets see 'THE TRAVEL AGENCY IN THE PARK'

In [24]:
data_dict['THE TRAVEL AGENCY IN THE PARK']

{'salary': 'NaN',
 'to_messages': 'NaN',
 'deferral_payments': 'NaN',
 'total_payments': 362096,
 'loan_advances': 'NaN',
 'bonus': 'NaN',
 'email_address': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'deferred_income': 'NaN',
 'total_stock_value': 'NaN',
 'expenses': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'exercised_stock_options': 'NaN',
 'from_messages': 'NaN',
 'other': 362096,
 'from_this_person_to_poi': 'NaN',
 'poi': False,
 'long_term_incentive': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'restricted_stock': 'NaN',
 'director_fees': 'NaN'}

We see some payments has been done to this 'THE TRAVEL AGENCY IN THE PARK', lets see in the net what is this.
Interesting, this is a kind of travel agency half owned by the sister of... Kenneth. "While Enron's employees were in theory permitted to book their business travel elsewhere, Sharon Lay's was the agency of choice" By Jyoti Thottam/Houston write

It will be removed later in the process as well as 'Total'

How many Poi do we have in the data set?

In [31]:
len(df_poi)

18

In [ ]:
Let see who are the those 18 Poi (Person of interest), their total payment, stock and emails exchange

In [55]:
poi = data_df.loc[:, 'poi'] == True
poi_df = data_df.loc[poi]

poi =poi_df.loc[:, ('poi','total_payments','total_stock_value',
                     'email_address', 'from_messages', 'to_messages' )]
poi.head(len(poi))

,poi,total_payments,total_stock_value,email_address,from_messages,to_messages
HANNON KEVIN P,True,288682,6391065,kevin.hannon@enron.com,32,1045
COLWELL WESLEY,True,1490344,698242,wes.colwell@enron.com,40,1758
RIEKER PAULA H,True,1099100,1918887,paula.rieker@enron.com,82,1328
KOPPER MICHAEL J,True,2652612,985032,michael.kopper@enron.com,NaN,NaN
SHELBY REX,True,2003885,2493616,rex.shelby@enron.com,39,225
DELAINEY DAVID W,True,4747979,3614261,david.delainey@enron.com,3069,3093
LAY KENNETH L,True,103559793,49110078,kenneth.lay@enron.com,36,4273
BOWEN JR RAYMOND M,True,2669589,252055,raymond.bowen@enron.com,27,1858
BELDEN TIMOTHY N,True,5501630,1110705,tim.belden@enron.com,484,7991
FASTOW ANDREW S,True,2424083,1794412,andrew.fastow@enron.com,NaN,NaN


We have 18 person of interes with their mails, but 4 has NaN in from poi and to poi messagess 

Lets check the people name in the entries are correct by checking if they are surname name initial as we see in the head of the data set. This will allow us to find further divergency not found until now.

In [23]:
import re
for index in data_df.index:
    if re.match('^[A-Z]+\s[A-Z]+(\s[A-Z])?$', index):
        continue
    else:
        print(index)

WALLS JR ROBERT H
SULLIVAN-SHAKLOVITZ COLLEEN
BOWEN JR RAYMOND M
OVERDYKE JR JERE C
PEREIRA PAULO V. FERRAZ
BLAKE JR. NORMAN P
THE TRAVEL AGENCY IN THE PARK
TOTAL
WHITE JR THOMAS E
WINOKUR JR. HERBERT S
GARLAND C KEVIN
YEAGER F SCOTT
DERRICK JR. JAMES V
DONAHUE JR JEFFREY M
GLISAN JR BEN F


Seams those names are right, we have some junior and some composed names.

In [ ]:
https://nbviewer.jupyter.org/github/vsvasconcelos/ML/blob/master/DocumentoProjeto.html

In [ ]:
data_dict_df_POI = data_dict_df[data_dict_df.poi==True]
data_dict_df_NPOI = data_dict_df[data_dict_df.poi==False]
mean_POI = pd.DataFrame(data_dict_df_POI.mean(), columns = ['POI'])
mean_NPOI = pd.DataFrame(data_dict_df_NPOI.mean(), columns = ['NPOI'])
df_mean = pd.concat([mean_POI, mean_NPOI], axis=1)
df_mean['POI_NPOI'] = df_mean['POI']/df_mean['NPOI']
df_mean.sort_values(by = "POI_NPOI", ascending=False)

In [ ]:
Later we will check with PCA if the machine agree with us 

Now lets pass to the next point: data check